## **3.2.1 - SVM without slack formulation - binary class**

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Load the Dataset
import numpy as np

file_path = '/content/drive/MyDrive/PRNN Assignment 2/multi_class_classification_data_group_25_train.txt'

# Assuming the first row is the header and the last column is the label
data = np.loadtxt(file_path, delimiter='\t', skiprows=1)

# Step 3: Split the Data into features and labels
X = data[:, :-1]  # All rows, all columns except the last one
y = data[:, -1]   # All rows, last column only

# Step 4: Split the Data into Training and Testing sets
# Decide on a split ratio, for example, 80% training and 20% testing
split_ratio = 0.8
split_index = int(X.shape[0] * split_ratio)

# Randomly shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split the data
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (11200, 10)
y_train shape: (11200,)
X_test shape: (2800, 10)
y_test shape: (2800,)


In [ ]:
y_train_transformed = y_train * 2 - 1  # This transforms 0 to -1 and 1 to 1
y_test_transformed = y_test * 2 - 1  # Same transformation for the test set

# Debug print statements to check if the transformation is correct
print("Transformed labels (train):", np.unique(y_train_transformed))
print("Transformed labels (test):", np.unique(y_test_transformed))

Transformed labels (train): [-1.  1.]
Transformed labels (test): [-1.  1.]


In [ ]:
import cvxpy as cp
import numpy as np

# Simple and clearly linearly separable dataset
X = np.array([[2, 2], [4, 4], [0, 1], [1, 3]])
y = np.array([1, 1, -1, -1])

# Variables for the optimization problem
n_samples, n_features = X.shape
w = cp.Variable(n_features)
b = cp.Variable()

# Objective function: 1/2 * ||w||^2
objective = cp.Minimize(0.5 * cp.norm(w, 2)**2)

# Constraints: y_i * (w^T * x_i + b) >= 1 for all samples
constraints = [y[i] * (X[i] @ w + b) >= 1 for i in range(n_samples)]

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Results
w_opt = w.value
b_opt = b.value

print(f"Optimal weight vector (w): {w_opt}")
print(f"Optimal bias (b): {b_opt}")
print("Solver status:", prob.status)

Optimal weight vector (w): [ 1.33333333 -0.66666667]
Optimal bias (b): -0.3333333363967996
Solver status: optimal


In [ ]:
import cvxpy as cp
import numpy as np

# Select the first 100 data points
X_subset = X_train[:100]
y_subset = y_train_transformed[:100]

# Number of samples and features in the subset
n_samples_subset, n_features_subset = X_subset.shape

# Optimization variables
w = cp.Variable(n_features_subset)
b = cp.Variable()

# Objective function
objective = cp.Minimize(0.5 * cp.norm(w, 2)**2)

# Constraints
constraints = [y_subset[i] * (X_subset[i] @ w + b) >= 1 for i in range(n_samples_subset)]

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Results
w_opt = w.value
b_opt = b.value

print(f"Optimal weight vector (w) for the 100-point subset: {w_opt}")
print(f"Optimal bias (b) for the 100-point subset: {b_opt}")
print("Solver status for the 100-point subset:", prob.status)

Optimal weight vector (w) for the 100-point subset: None
Optimal bias (b) for the 100-point subset: None
Solver status for the 100-point subset: infeasible


## **3.2.2 - SVM with slack formulation - binary class**

In [ ]:
import cvxpy as cp
import numpy as np

# Assuming X_train and y_train_transformed are your full dataset and labels
n_samples, n_features = X_train.shape

# Variables for optimization
w = cp.Variable(n_features)
b = cp.Variable()
xi = cp.Variable(n_samples)  # Slack variables for the full dataset

# Regularization parameter
C = .1

# Objective function for the full dataset
objective = cp.Minimize(0.5 * cp.norm(w, 2)**2 + C * cp.sum(xi))

# Constraints for the full dataset
constraints = [y_train_transformed[i] * (X_train[i] @ w + b) >= 1 - xi[i] for i in range(n_samples)] + [xi[i] >= 0 for i in range(n_samples)]

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Results for the full dataset
w_opt_full = w.value
b_opt_full = b.value

print(f"Optimal weight vector (w) for the full dataset: {w_opt_full}")
print(f"Optimal bias (b) for the full dataset: {b_opt_full}")
print("Solver status for the full dataset:", prob.status)

Optimal weight vector (w) for the full dataset: [-0.16792165  0.31727058 -0.54082606 -0.1616919   0.14615286  0.27779174
  0.3074921  -0.5230517  -0.68889371 -0.30658345]
Optimal bias (b) for the full dataset: 0.6057291430156447
Solver status for the full dataset: optimal


**Test**

In [ ]:
n_test_samples, _ = X_test.shape

# Calculate the decision function for each test sample
decision_function = X_test.dot(w_opt_full) + b_opt_full

# Make predictions based on the sign of the decision function
predictions = np.sign(decision_function)

# y_test_transformed are true labels for the test dataset
# Convert predictions from -1,1 to 0,1 if original labels were 0 and 1
predictions_transformed = (predictions + 1) // 2

# Calculate accuracy
accuracy = np.mean(predictions_transformed == y_test_transformed)
print(f"Model accuracy on the test dataset: {accuracy * 100:.2f}%")

Model accuracy on the test dataset: 34.14%


In [ ]:
# Calculate the decision function for each training sample
decision_function_train = X_train.dot(w_opt_full) + b_opt_full

# Make predictions based on the sign of the decision function
predictions_train = np.sign(decision_function_train)

# y_train_transformed are true labels for the training dataset
# Convert predictions from -1,1 to 0,1 if original labels were 0 and 1
predictions_train_transformed = (predictions_train + 1) // 2

# Calculate accuracy on training data
accuracy_train = np.mean(predictions_train_transformed == y_train_transformed)
print(f"Model accuracy on the training dataset: {accuracy_train * 100:.2f}%")

Model accuracy on the training dataset: 33.66%


**Linear kernel**

In [ ]:
import numpy as np
from scipy.optimize import minimize

n_features = X_subset.shape[1]

# Objective function for the primal problem
def svm_primal_objective(params):
    w = params[:-1]
    b = params[-1]
    regularization = 0.5 * np.sum(w ** 2)
    hinge_loss = np.sum(np.maximum(0, 1 - y_subset * (X_subset.dot(w) + b)))
    return regularization + C * hinge_loss

# Regularization parameter C
C = 1

# Initial guess for w and b
initial_params = np.zeros(n_features + 1)

# Optimization
result = minimize(svm_primal_objective, initial_params, method='SLSQP')

if result.success:
    optimized_params = result.x
    w_opt = optimized_params[:-1]
    b_opt = optimized_params[-1]
    print(f"Optimal weight vector (w): {w_opt}")
    print(f"Optimal bias (b): {b_opt}")
else:
    print("Optimization was not successful. Check the objective function and constraints.")

Optimal weight vector (w): [ 0.30952046  0.55017813 -0.2130002  -0.49864981  0.54300217  0.25091342
 -0.54073784 -0.62652359 -0.76701829 -0.04458616]
Optimal bias (b): 0.18462387572828234


In [ ]:

# Calculate the decision values for each test sample
decision_values = X_test.dot(w_opt) + b_opt

# Predict class labels based on the sign of the decision values
predictions = np.sign(decision_values)

# Convert predictions from -1,1 to the original label space if necessary

# Calculate accuracy
accuracy = np.mean(predictions == y_test_transformed)
print(f"Model accuracy on the test dataset: {accuracy * 100:.2f}%")

Model accuracy on the test dataset: 62.04%


**Polynomial kernel**

In [ ]:
def polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1):
    if Y is None:
        Y = X
    if gamma is None:
        gamma = 1.0 / X.shape[1]  # Default value of gamma is 1 / n_features

    K = (gamma * np.dot(X, Y.T) + coef0) ** degree
    return K

In [ ]:
import cvxpy as cp
import numpy as np

# Define polynomial kernel
def polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1):
    if Y is None:
        Y = X
    if gamma is None:
        gamma = 1.0 / X.shape[1]
    K = (gamma * np.dot(X, Y.T) + coef0) ** degree
    return K

degree = 3
gamma = None  # Automatically set to 1/n_features within the function
coef0 = 1

# Compute the kernel matrix
K = polynomial_kernel(X_subset, degree=degree, gamma=gamma, coef0=coef0)

# Symmetrize the matrix used in quadratic form
Q = np.multiply(np.outer(y_subset, y_subset), K)
Q_sym = 0.5 * (Q + Q.T)  # Symmetrizing the matrix

n_samples = X_subset.shape[0]
alpha = cp.Variable(n_samples)

# Dual problem objective with corrected matrix
objective = cp.Maximize(
    cp.sum(alpha) - 0.5 * cp.quad_form(alpha, Q_sym)
)

constraints = [
    alpha >= 0,
    alpha <= 1,  # Assuming C=1; adjust as needed
    cp.sum(cp.multiply(alpha, y_subset)) == 0
]

# Solve the problem
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Check if the optimization was successful
if result is not None:
    optimized_alphas = alpha.value
    print("Optimization successful.")
else:
    print("Optimization failed.")

Optimization successful.


In [ ]:
# Indices of support vectors
sv_indices = optimized_alphas > 1e-5

# Alphas for support vectors
alphas_sv = optimized_alphas[sv_indices]

# Support vectors and their labels
X_sv = X_subset[sv_indices]
y_sv = y_subset[sv_indices]

# Calculate bias using support vectors
b = np.mean(
    [y_k - np.sum(alphas_sv * y_sv * polynomial_kernel(X_sv, x_k[np.newaxis, :], degree=degree, gamma=gamma, coef0=coef0).flatten())
     for y_k, x_k in zip(y_sv, X_sv)]
)

print(f"Optimal bias (b): {b}")

Optimal bias (b): 0.37653078023302344


In [ ]:
def predict(X_new, X_sv, y_sv, alphas_sv, b, kernel):
    decision_function = np.sum(alphas_sv * y_sv * kernel(X_sv, X_new), axis=0) + b
    return np.sign(decision_function)

In [ ]:
def predict(X_new, X_sv, y_sv, alphas_sv, b, kernel, degree=3, gamma=None, coef0=1):
    """
    Predict the class labels for new data points with the trained SVM model.

    :param X_new: New data points to predict
    :param X_sv: Support vectors from the training data
    :param y_sv: Labels of the support vectors
    :param alphas_sv: Optimized alpha values for the support vectors
    :param b: Calculated bias of the SVM model
    :param kernel: Kernel function used by the SVM model
    :param degree: Degree for the polynomial kernel
    :param gamma: Scale parameter for the polynomial kernel
    :param coef0: Independent term in polynomial kernel
    :return: Predicted class labels for X_new
    """
    K = kernel(X_sv, X_new, degree=degree, gamma=gamma, coef0=coef0)
    decision_values = np.dot(alphas_sv * y_sv, K) + b
    return np.sign(decision_values)

predictions = predict(X_test, X_sv, y_sv, alphas_sv, b, polynomial_kernel, degree=degree, gamma=gamma, coef0=coef0)

# Calculate accuracy
accuracy = np.mean(predictions == y_test_transformed)
print(f"Model accuracy on the test dataset: {accuracy * 100:.2f}%")

Model accuracy on the test dataset: 66.64%


## **3.2.3 - SVM without slack formulation - multiclass**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

file_path = '/content/drive/MyDrive/PRNN Assignment 2/multi_class_classification_data_group_25_train2.txt'

# Skipping the first row since it contains headers.
data = np.genfromtxt(file_path, delimiter='\t', skip_header=1)

print("Data loaded successfully")

Data loaded successfully


In [ ]:
# Ensure reproducibility
np.random.seed(42)

# Shuffle the data
np.random.shuffle(data)

# Define the split ratio
split_ratio = 0.8

# Calculate the split index
split_index = int(data.shape[0] * split_ratio)

# Split the data
train_data = data[:split_index]
test_data = data[split_index:]

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

Training data shape: (56000, 26)
Testing data shape: (14000, 26)


In [ ]:
import numpy as np

# Extracting features and labels
features = train_data[:, :-1]  # Exclude the last column, which is the label
labels = train_data[:, -1]  # The last column is the label

# Debug print before normalization
print("Before normalization:")
print("Features shape:", features.shape)
print("First 5 rows of features:\n", features[:5, :])
print("Labels shape:", labels.shape)
print("First 5 labels:", labels[:5])

# Normalize features
mean = np.mean(features, axis=0)
std = np.std(features, axis=0)
normalized_features = (features - mean) / std

# Debug print after normalization
print("\nAfter normalization:")
print("Normalized features shape:", normalized_features.shape)
print("First 5 rows of normalized features:\n", normalized_features[:5, :])

# Confirming mean ~ 0 and std ~ 1
print("\nMean of normalized features (should be close to 0):\n", np.mean(normalized_features, axis=0)[:5])
print("Standard deviation of normalized features (should be close to 1):\n", np.std(normalized_features, axis=0)[:5])

Before normalization:
Features shape: (56000, 25)
First 5 rows of features:
 [[ 0.88231309  2.060794   -0.03727296  0.3633859   0.712504   -1.54848548
   0.74139339  0.50145251  0.82316467  0.50114331  0.30472922  2.93303774
   0.11544043  0.57320856  0.30664962 -1.35473103  0.01571383 -0.37885504
   0.90970749  0.76453455  0.88939557  0.08771643  0.46922044  1.98547878
   0.9951316 ]
 [-0.60835611 -0.42902146 -0.02019832 -1.19107419  0.35161801  0.02221423
   1.00612765  2.57896898 -0.21904903 -0.08734955 -0.88490681  1.4437579
   0.3738467   0.19311729  0.35989156  1.22988802  1.67289023  0.54611338
   0.13648412  0.89357358  0.16569693 -0.37054891 -0.34939985  0.61531351
   2.05849387]
 [ 0.19217755  0.62725995 -0.37947646  1.77115989  0.23299918 -0.08514341
   0.3335442   0.32321727  0.88810755  0.60299201  0.53586898 -0.44658212
   0.37783549  0.30922605  1.3429528   0.0675     -0.46004075  0.38989001
  -0.01722178 -0.09530088 -0.86920728 -0.21681016  1.69519322  0.70981577
   0.7

In [ ]:
def prepare_binary_labels(labels, class_label):
    """
    This function prepares binary labels for one-vs-rest classification.
    All instances of the specified class_label are marked as 1, and all other instances are marked as 0.

    :param labels: The array of multi-class labels.
    :param class_label: The current class for which the binary labels are being prepared.
    :return: An array of binary labels.
    """
    binary_labels = np.where(labels == class_label, 1, 0)
    return binary_labels

# Example usage for class 0
class_0_labels = prepare_binary_labels(labels, 0)

# Debug print to verify binary labels for class 0
print("First 20 labels (original):", labels[:20])
print("First 20 binary labels for class 0:", class_0_labels[:20])

First 20 labels (original): [7. 9. 6. 8. 5. 2. 8. 6. 1. 0. 1. 6. 3. 7. 5. 3. 1. 8. 3. 2.]
First 20 binary labels for class 0: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


To focus on computational efficiency and demonstrate the concept with limited resources, we'll adjust the script to train SVM classifiers for each class using only the first 100 data points from the training dataset.

In [ ]:
import cvxpy as cp
import numpy as np

# Limit to the first 100 data points for demonstration
X_train_subset = normalized_features[:100]
y_train_subset = labels[:100]

num_classes = len(np.unique(y_train_subset))  # Number of unique classes

# Initialize lists to store the optimal weights and biases for each SVM
optimal_weights = []
optimal_biases = []

for class_label in range(num_classes):
    print(f"Training SVM for class {class_label} vs rest")

    # Prepare binary labels for the current class vs all others
    y_train_transformed = np.where(y_train_subset == class_label, 1, -1)

    # Optimization variables
    w = cp.Variable(X_train_subset.shape[1])
    b = cp.Variable()

    # Objective function
    objective = cp.Minimize(0.5 * cp.norm(w, 2)**2)

    # Constraints
    constraints = [y_train_transformed[i] * (X_train_subset[i] @ w + b) >= 1 for i in range(100)]

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Store the results
    optimal_weights.append(w.value)
    optimal_biases.append(b.value)

    print(f"Finished training SVM for class {class_label} vs rest. Solver status: {prob.status}")

print("Training complete for all classes.")

Training SVM for class 0 vs rest
Finished training SVM for class 0 vs rest. Solver status: optimal
Training SVM for class 1 vs rest
Finished training SVM for class 1 vs rest. Solver status: infeasible
Training SVM for class 2 vs rest
Finished training SVM for class 2 vs rest. Solver status: optimal
Training SVM for class 3 vs rest
Finished training SVM for class 3 vs rest. Solver status: optimal
Training SVM for class 4 vs rest
Finished training SVM for class 4 vs rest. Solver status: infeasible
Training SVM for class 5 vs rest
Finished training SVM for class 5 vs rest. Solver status: optimal
Training SVM for class 6 vs rest
Finished training SVM for class 6 vs rest. Solver status: optimal
Training SVM for class 7 vs rest


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Finished training SVM for class 7 vs rest. Solver status: optimal_inaccurate
Training SVM for class 8 vs rest
Finished training SVM for class 8 vs rest. Solver status: infeasible
Training SVM for class 9 vs rest
Finished training SVM for class 9 vs rest. Solver status: optimal
Training complete for all classes.


## **3.2.4 - SVM with slack formulation - multiclass**

In [ ]:
import cvxpy as cp
import numpy as np


X_train = normalized_features[:100]  # Using the first 100 data points
y_train = labels[:100]

num_classes = len(np.unique(y_train))  # Number of unique classes

# Initialize lists to store the optimal weights and biases for each SVM
optimal_weights = []
optimal_biases = []

# Regularization parameter C
C = 1

for class_label in range(num_classes):
    print(f"Training SVM with slacks for class {class_label} vs rest")

    # Prepare binary labels for the current class vs all others
    y_train_transformed = np.where(y_train == class_label, 1, -1)

    # Optimization variables
    w = cp.Variable(X_train.shape[1])
    b = cp.Variable()
    xi = cp.Variable(X_train.shape[0])  # Slack variable for each data point

    # Objective function
    objective = cp.Minimize(0.5 * cp.norm(w, 2)**2 + C * cp.sum(xi))

    # Constraints
    constraints = [y_train_transformed[i] * (X_train[i] @ w + b) >= 1 - xi[i] for i in range(X_train.shape[0])]
    constraints += [xi[i] >= 0 for i in range(X_train.shape[0])]  # Slack variables must be non-negative

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Store the results
    optimal_weights.append(w.value)
    optimal_biases.append(b.value)

    print(f"Finished training SVM with slacks for class {class_label} vs rest. Solver status: {prob.status}")

print("Training complete for all classes with slacks.")

Training SVM with slacks for class 0 vs rest
Finished training SVM with slacks for class 0 vs rest. Solver status: optimal
Training SVM with slacks for class 1 vs rest
Finished training SVM with slacks for class 1 vs rest. Solver status: optimal
Training SVM with slacks for class 2 vs rest
Finished training SVM with slacks for class 2 vs rest. Solver status: optimal
Training SVM with slacks for class 3 vs rest
Finished training SVM with slacks for class 3 vs rest. Solver status: optimal
Training SVM with slacks for class 4 vs rest
Finished training SVM with slacks for class 4 vs rest. Solver status: optimal
Training SVM with slacks for class 5 vs rest
Finished training SVM with slacks for class 5 vs rest. Solver status: optimal
Training SVM with slacks for class 6 vs rest
Finished training SVM with slacks for class 6 vs rest. Solver status: optimal
Training SVM with slacks for class 7 vs rest
Finished training SVM with slacks for class 7 vs rest. Solver status: optimal
Training SVM wit

Accuracy

In [ ]:
# Extracting test features and labels
test_features = test_data[:, :-1]  # Exclude the last column, which is the label
test_labels = test_data[:, -1]  # The last column is the label

# Normalize test features using mean and std from the training features
normalized_test_features = (test_features - mean) / std

# Ensure the predict function is defined as before
def predict(X, weights, biases):
    # Calculate the score for each classifier
    scores = np.array([X.dot(w) + b for w, b in zip(weights, biases)])
    # Determine the predicted class with the highest score
    predicted_classes = np.argmax(scores, axis=0)
    return predicted_classes

# Predict labels for the normalized test features
predicted_labels = predict(normalized_test_features, optimal_weights, optimal_biases)

# Calculate accuracy by comparing predicted labels with actual test labels
accuracy = np.mean(predicted_labels == test_labels)
print(f"Accuracy on the test data: {accuracy:.4f}")

Accuracy on the test data: 0.1484


**Linear Kernels**

In [ ]:
import numpy as np
from scipy.optimize import minimize

num_classes = len(np.unique(labels))

optimal_weights = []
optimal_biases = []

C = 0.1  # Regularization parameter

for class_label in range(num_classes):
    print(f"\nTraining for class {class_label} vs rest")

    y_subset = np.where(labels == class_label, 1, -1)
    X_subset = normalized_features

    def svm_primal_objective(params):
        w = params[:-1]
        b = params[-1]
        regularization = 0.5 * np.sum(w ** 2)
        hinge_loss = np.sum(np.maximum(0, 1 - y_subset * (X_subset.dot(w) + b)))
        return regularization + C * hinge_loss

    n_features = X_subset.shape[1]
    initial_params = np.zeros(n_features + 1)

    # Optimization with increased iterations limit
    result = minimize(svm_primal_objective, initial_params, method='SLSQP', bounds=[(None, None)] * (n_features + 1), options={'disp': True, 'maxiter': 1000})  # Increased maxiter

    if result.success:
        optimized_params = result.x
        w_opt = optimized_params[:-1]
        b_opt = optimized_params[-1]
        optimal_weights.append(w_opt)
        optimal_biases.append(b_opt)
        print(f"Class {class_label}: Optimization successful.")
    else:
        print(f"Class {class_label}: Optimization was not successful. Message: {result.message}")


Training for class 0 vs rest
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1121.8017948380716
            Iterations: 111
            Function evaluations: 3247
            Gradient evaluations: 111
Class 0: Optimization successful.

Training for class 1 vs rest
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1119.6013303331617
            Iterations: 103
            Function evaluations: 3031
            Gradient evaluations: 103
Class 1: Optimization successful.

Training for class 2 vs rest
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1111.8013971304365
            Iterations: 111
            Function evaluations: 3242
            Gradient evaluations: 111
Class 2: Optimization successful.

Training for class 3 vs rest
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1118.6017522874613
            Iterations: 108

In [ ]:

# Extracting test features and labels
test_features = test_data[:, :-1]  # Exclude the last column, which is the label
test_labels = test_data[:, -1]  # The last column is the label

# Normalize test features using mean and std from the training features
normalized_test_features = (test_features - mean) / std

# Prediction function adapted for the trained SVM models
def predict(X, weights, biases):
    # Calculate the score for each classifier
    scores = np.array([X.dot(w) + b for w, b in zip(weights, biases)])
    # Determine the predicted class with the highest score for each sample
    predicted_classes = np.argmax(scores, axis=0)
    return predicted_classes

# Predict labels for the normalized test features using the new model
predicted_labels = predict(normalized_test_features, optimal_weights, optimal_biases)

# Calculate accuracy by comparing predicted labels with actual test labels
accuracy = np.mean(predicted_labels == test_labels)
print(f"Accuracy on the test data: {accuracy:.4f}")

Accuracy on the test data: 0.1585


**Polynomial kernel**

In [ ]:
def polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1):
    if Y is None:
        Y = X
    if gamma is None:
        gamma = 1.0 / X.shape[1]  # Default value of gamma is 1 / n_features

    K = (gamma * np.dot(X, Y.T) + coef0) ** degree
    return K

In [ ]:
import numpy as np
import cvxpy as cp

# Assuming normalized_features and labels are defined
# Use a small subset for debugging to prevent crashing
X_debug = normalized_features[:100]
y_debug = np.where(labels[:100] == 0, 1, -1)  # Example: Class 0 vs rest

# Define polynomial kernel function
def polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1):
    if Y is None:
        Y = X
    if gamma is None:
        gamma = 1.0 / X.shape[1]
    K = (gamma * np.dot(X, Y.T) + coef0) ** degree
    return K

# Compute the kernel matrix for the debug subset
K_debug = polynomial_kernel(X_debug, degree=2, gamma=None, coef0=1)  # Using a simpler kernel for debugging

# Symmetrize the kernel matrix used in quadratic form
Q_debug = np.multiply(np.outer(y_debug, y_debug), K_debug)
Q_sym_debug = 0.5 * (Q_debug + Q_debug.T)  # Symmetrizing the matrix

n_samples_debug = X_debug.shape[0]
alpha_debug = cp.Variable(n_samples_debug)

# Dual problem objective
objective_debug = cp.Maximize(
    cp.sum(alpha_debug) - 0.5 * cp.quad_form(alpha_debug, Q_sym_debug)
)

# Constraints
constraints_debug = [
    alpha_debug >= 0,
    alpha_debug <= 1,  # Assuming C=1 for debugging
    cp.sum(cp.multiply(alpha_debug, y_debug)) == 0
]

# Solve the problem for debugging
prob_debug = cp.Problem(objective_debug, constraints_debug)
result_debug = prob_debug.solve(solver=cp.SCS, verbose=True)

# Check if the optimization was successful
if result_debug is not None:
    print("Optimization successful.")
    optimized_alphas_debug = alpha_debug.value
    # Additional steps to calculate support vectors, weights, and bias would go here
else:
    print("Optimization failed.")

                                     CVXPY                                     
                                     v1.3.3                                    
(CVXPY) Mar 27 04:38:44 PM: Your problem has 100 variables, 3 constraints, and 0 parameters.
(CVXPY) Mar 27 04:38:44 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 27 04:38:44 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 27 04:38:44 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 27 04:38:44 PM: Compiling problem (target solver=SCS).
(CVXPY) Mar 27 04:38:44 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Cone

In [ ]:
# Threshold for considering an alpha to correspond to a support vector
alpha_threshold = 1e-4

# Identifying the support vectors
support_vector_indices = np.where(optimized_alphas_debug > alpha_threshold)[0]
support_vectors = X_debug[support_vector_indices]
support_alphas = optimized_alphas_debug[support_vector_indices]
support_labels = y_debug[support_vector_indices]

print(f"Number of support vectors: {len(support_vector_indices)}")

Number of support vectors: 40


In [ ]:
import numpy as np

# Define the polynomial kernel parameters
degree = 3
coef0 = 1
n_features = X_debug.shape[1]
gamma = 1.0 / n_features  # Define gamma as 1/n_features if not defined

# Identifying the support vectors and their properties
alpha_threshold = 1e-4  # Threshold for alpha to identify support vectors
support_vector_indices = np.where(optimized_alphas_debug > alpha_threshold)[0]
support_vectors = X_debug[support_vector_indices]
support_alphas = optimized_alphas_debug[support_vector_indices]
support_labels = y_debug[support_vector_indices]

# Compute the kernel between support vectors and all vectors
K_support_all = polynomial_kernel(support_vectors, X_debug, degree=degree, gamma=gamma, coef0=coef0)

# Compute the bias term (b)
# Simplified approach: average over all support vectors
sum_b = 0
for i in range(len(support_vectors)):
    sum_b += support_labels[i] - np.sum(support_alphas * support_labels * K_support_all[:, i])
b = sum_b / len(support_vectors)

print(f"Bias (b): {b}")

Bias (b): -0.5981252247196116


In [ ]:

# Extract test features and labels
test_features = test_data[:, :-1]  # Exclude the last column, which is the label
test_labels = test_data[:, -1]  # The last column is the label

# Normalize test features using the mean and std from the training features
normalized_test_features = (test_features - mean) / std

In [ ]:
def predict_with_kernel(X):
    # Compute the polynomial kernel between X and the support vectors
    K_test_support = polynomial_kernel(X, support_vectors, degree=degree, gamma=gamma, coef0=coef0)

    # Calculate the decision function for each sample in X
    decision_values = np.dot(K_test_support, (support_alphas * support_labels)) + b

    # Assign class labels based on the sign of the decision function
    return np.sign(decision_values)

# Now that 'normalized_test_features' is defined, predict labels for the test set
predicted_labels = predict_with_kernel(normalized_test_features)

# Convert predicted labels from 1, -1 to the original class labels for binary classification
predicted_labels_converted = np.where(predicted_labels == 1, 0, 1)  # Assuming class 0 vs rest, adjust as necessary

# Calculate accuracy by comparing predicted labels with actual test labels
accuracy = np.mean(predicted_labels_converted == test_labels)
print(f"Accuracy on the test data: {accuracy:.4f}")

Accuracy on the test data: 0.1065


**Radial Basis Function (RBF) Kernel**

In [ ]:
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import norm

subset_size = 100  # Adjust based on computational resources
np.random.seed(42)  # For reproducibility
indices = np.random.choice(len(normalized_features), size=subset_size, replace=False)

X_subset = normalized_features[indices]
y_subset = np.where(labels[indices] == class_label, 1, -1)

# Define the RBF kernel function
def rbf_kernel(x1, x2, gamma=1.0):
    dist_sq = norm(x1[:, None] - x2, axis=2) ** 2
    return np.exp(-gamma * dist_sq)

# Gamma for the RBF kernel
gamma = 0.1

# Regularization parameter C
C = 1.0  # Adjust based on your dataset

# Dual problem objective function to be minimized
def dual_objective(alpha):
    K = rbf_kernel(X_subset, X_subset, gamma=gamma)
    half_quad_form = 0.5 * np.dot(alpha, np.dot(alpha, K * y_subset[:, None] * y_subset[None, :]))
    return half_quad_form - np.sum(alpha)

# Constraint ensuring that the sum of alpha_i * y_i = 0
cons = ({'type': 'eq', 'fun': lambda alpha: np.dot(alpha, y_subset)})

# Bounds for alpha_i, 0 <= alpha_i <= C for all i
bounds = [(0, C) for _ in range(subset_size)]

# Initial guess for alpha
initial_alpha = np.zeros(subset_size)

# Solve the quadratic optimization problem
result = minimize(dual_objective, initial_alpha, method='SLSQP', bounds=bounds, constraints=cons)

if result.success:
    alphas = result.x
    sv = alphas > 1e-5  # Threshold for determining support vectors
    support_vectors = X_subset[sv]
    support_labels = y_subset[sv]
    support_alphas = alphas[sv]
    # Calculate the bias term
    K_sv = rbf_kernel(support_vectors, support_vectors, gamma)
    b = np.mean([y_k - np.sum(support_alphas * support_labels * K_sv[i]) for i, y_k in enumerate(support_labels)])
    print("Optimization successful.")
else:
    print("Optimization was not successful. Message:", result.message)

Optimization successful.


In [ ]:
# Print the optimized parameters of the SVM
print("Optimized alpha values for support vectors:\n", support_alphas)

# The support vectors themselves
print("\nSupport vectors:\n", support_vectors)

# The labels for the support vectors
print("\nLabels for support vectors:\n", support_labels)

# Bias term
print("\nBias term (b):", b)

Optimized alpha values for support vectors:
 [0.24778312 0.05745584 0.31171832 1.         1.         0.1332226
 0.08374385 1.         0.09487018 0.17078726 1.         0.17048859
 0.13296417 0.12819877 1.         0.25003186 0.16971832 0.08200471
 0.16354856 0.28470993 0.12634862 0.19433413 0.13222096 0.147097
 0.16172827 0.16115765 0.13933604 0.05554849 0.13756415 0.14606729
 1.         0.20516658 0.17581577 0.24459862 0.12832111 0.16080271
 0.19880502 0.16646328 0.13503006 0.14590697 0.17372692 0.12522789
 0.14908998 0.12285731 0.1579307  0.1478242  0.10099618 0.208695
 0.05883275 0.13622678 0.16639841 0.12196853 0.20160277 0.14035349
 1.         0.16780417 0.12524661 0.14651123 0.06847318 0.10788035
 0.18469475 0.19747455 0.11395699 0.28488852 0.16264671 0.16306459
 0.08076093 0.04688382 0.22925615 0.10118664 0.16742957 1.
 0.1385258  0.05775931 0.13634216 1.         0.22700654 0.18094819
 0.18115859 0.15297201 0.15526631 1.         1.         0.11352388
 0.14505836 0.14018757 0.12897

In [ ]:

# Extract test features and labels
test_features = test_data[:, :-1]  # Exclude the last column, which is the label
test_labels = test_data[:, -1]  # The last column is the label

# Normalize test features using the mean and std from the training features
normalized_test_features = (test_features - mean) / std

In [ ]:
def predict_with_kernel(X, support_vectors, support_labels, support_alphas, b, gamma):
    # Compute the RBF kernel between X and the support vectors
    K_test_support = np.array([rbf_kernel(x[np.newaxis, :], support_vectors, gamma=gamma) for x in X])

    # Calculate the decision function for each sample in X
    decision_values = np.dot(K_test_support, (support_alphas * support_labels)) + b

    # Assign class labels based on the sign of the decision function
    return np.sign(decision_values)

# Predict labels for the test set
predicted_labels = predict_with_kernel(normalized_test_features, support_vectors, support_labels, support_alphas, b, gamma)

predicted_labels_converted = np.where(predicted_labels == 1, class_label, 0)  # Assuming class 0 vs rest as an example

# Calculate accuracy by comparing predicted labels with actual test labels
accuracy = np.mean(predicted_labels_converted == test_labels)
print(f"Accuracy on the test data: {accuracy:.4f}")

Accuracy on the test data: 0.0984
